In [3]:
import sqlite3

In [4]:
connection = sqlite3.connect('parent.db')

In [5]:
!ls

300features_40minwords_10context
500features_5minwords_4context
Capture.PNG
Capture1.PNG
Capture3.PNG
Capture4.PNG
Capture5.PNG
Capture6.PNG
Part 2.ipynb
SQL.ipynb
Untitled.png
parent.db
prideNprejudice.csv


In [6]:
cursor = connection.cursor()

In [7]:
cursor.execute("DROP TABLE IF EXISTS parent;")

In [8]:
# Create the table
sql_command = '''
CREATE TABLE parent AS
  SELECT "abraham" AS parent, "barack" AS child UNION
  SELECT "abraham",           "clinton"         UNION
  SELECT "delano",            "herbert"         UNION
  SELECT "fillmore",          "abraham"         UNION
  SELECT "fillmore",          "delano"          UNION
  SELECT "fillmore",          "grover"          UNION
  SELECT "eisenhower",        "fillmore";
'''

cursor.execute(sql_command)

In [9]:
connection.commit()
!ls

300features_40minwords_10context
500features_5minwords_4context
Capture.PNG
Capture1.PNG
Capture3.PNG
Capture4.PNG
Capture5.PNG
Capture6.PNG
Part 2.ipynb
SQL.ipynb
Untitled.png
parent.db
prideNprejudice.csv


## Q1 Simple selects

In [10]:
import pandas as pd
pd.read_sql_query('SELECT * FROM parent;', connection)

,parent,child
0,abraham,barack
1,abraham,clinton
2,delano,herbert
3,eisenhower,fillmore
4,fillmore,abraham
5,fillmore,delano
6,fillmore,grover


In [11]:
cursor.execute('SELECT * FROM parent') 

print("fetchall:") 

result = cursor.fetchall() 

for r in result:

    print(r) 

fetchall:
('abraham', 'barack')
('abraham', 'clinton')
('delano', 'herbert')
('eisenhower', 'fillmore')
('fillmore', 'abraham')
('fillmore', 'delano')
('fillmore', 'grover')


In [12]:
df = pd.read_sql_query('SELECT * '
                       'FROM parent ' 
                       'WHERE parent = "abraham" '
                       , connection)

df

,parent,child
0,abraham,barack
1,abraham,clinton


In [13]:
df1 = pd.read_sql_query("SELECT child FROM parent WHERE child LIKE '%e%' " , connection)
df1

,child
0,herbert
1,fillmore
2,delano
3,grover


In [14]:
df2 = pd.read_sql_query("SELECT DISTINCT parent FROM parent ORDER BY parent DESC", connection)
df2

,parent
0,fillmore
1,eisenhower
2,delano
3,abraham


In [15]:
df3 = pd.read_sql_query("SELECT child FROM parent WHERE parent = 'abraham'", connection)

df3



,child
0,barack
1,clinton


## Q2 Joints

In [16]:
cursor.execute('DROP TABLE IF EXISTS dogs')
sql_command = '''
CREATE TABLE dogs AS
  SELECT "abraham" AS name, "long" AS fur UNION
  SELECT "barack",          "short"       UNION
  SELECT "clinton",         "long"        UNION
  SELECT "delano",          "long"        UNION
  SELECT "eisenhower",      "short"       UNION
  SELECT "fillmore",        "curly"       UNION
  SELECT "grover",          "short"       UNION
  SELECT "herbert",         "curly";
'''
cursor.execute(sql_command)

In [17]:
connection.commit()

### Count the Number of short haired dogs

In [18]:
connection = sqlite3.connect('parent.db')
cursor = connection.cursor()
fur = cursor.execute('SELECT COUNT (*) ' 
                    'FROM dogs '
                    'WHERE fur = "short" ')

for row in fur.fetchall():
    print(row)

(3,)


### Join the table parents and dogs and select the parents of curly dogs

In [20]:
df5 = pd.read_sql_query("SELECT parent FROM parent JOIN dogs ON parent.child = dogs.name WHERE dogs.fur = 'curly' ", connection)
df5

,parent
0,eisenhower
1,delano


### Join tables parents and dogs, and select the parents and children that some have fur

In [21]:
furtype = cursor.execute('SELECT parent, child '
 'FROM parent '
'JOIN dogs a ON a.name = parent.child '
'JOIN dogs b ON b.name = parent.parent '
'WHERE a.fur = b.fur'
 )
for row in furtype.fetchall():
 print(row)

('abraham', 'clinton')


In [22]:
connection.commit()
connection.close()

# Q3 Aggregate function, numerical logic and function

In [23]:
connection = sqlite3.connect('parent.db')
cursor = connection.cursor()

In [24]:
#setup
cursor.execute('DROP TABLE IF EXISTS animals')
sql_command = """create table animals as
 select "dog" as kind, 4 as legs, 20 as weight union
 select "cat" , 4 , 10 union
 select "ferret" , 4 , 10 union
 select "parrot" , 2 , 6 union
 select "penguin" , 2 , 10 union
select "t-rex" , 2 , 12000;"""
cursor.execute(sql_command)

In [25]:
connection.commit()
connection.close()

### 1. Select the animal with the minimum weight. Display kind and min weight.

In [26]:
connection = sqlite3.connect('parent.db')
cursor = connection.cursor()

In [27]:
weight = cursor.execute('SELECT kind, MIN(weight) '
 'FROM animals ')
for row in weight.fetchall():
 print(row)

('parrot', 6)


### 2. Use the aggregate function AVG to display a table with the average number of legs and average weight.

In [28]:
avgweight = cursor.execute('SELECT AVG(legs), AVG(weight) '
 'FROM animals ')
for row in avgweight.fetchall():
 print(row)

(3.0, 2009.3333333333333)


### 3. SELECT the animal kind(s) that have more than two legs, but weighs less than 20. Display kind, weight, legs.


In [30]:
legs = cursor.execute('SELECT kind, weight, legs '
 'FROM animals '
'WHERE legs > 2 '
'AND weight < 20')
for row in legs.fetchall():
 print(row)

('cat', 10, 4)
('ferret', 10, 4)


### 4. SELECT the average weight for all the animals with 2 legs and the animals with 4 legs(by using GROUP BY).

In [31]:
weight = cursor.execute('SELECT AVG(weight) '
 'FROM animals '
'GROUP BY legs')
for row in weight.fetchall():
 print(row)

(4005.3333333333335,)
(13.333333333333334,)
